ID #1: 319238119

ID #2: 305560070

In [3]:
'''

Create local CSV file “mydata.csv” with 1000000 rows with columns (id, fruit, price, color). Use
random value for rows, where fruit has one of the values ['Orange', 'Grape', 'Apple', 'Banana',
'Pineapple', 'Avocado'] and colors are ['Red', 'Green', 'Yellow', 'Blue']. Price should be random
integer between 10 and 100. Filed id should be an index number starting from 1.

'''

import csv
import random
import pandas as pd

random.seed(42)

index = range(1, 1000001)
price = range(10, 101)

fruit = [random.choice(['Orange', 'Grape', 'Apple', 'Banana', 'Pineapple', 'Avocado']) for i in index]
color = [random.choice(['Red', 'Green', 'Yellow', 'Blue']) for i in index]
price = [random.choice(price) for i in index]

df = pd.DataFrame({'id' : index, 'fruit' : fruit, 'price' : price, 'color' : color})
df.to_csv('mydata.csv', index=False)

print('Done')

Done


In [4]:
''' 
Task 1 Question 1 

Write Python code to create SQLite database “mydb.db” and create a table “mydata” with the schema of the “mydata.csv”
'''

import sqlite3

conn = sqlite3.connect(r'mydb.db')
cur = conn.cursor()
cur.execute('CREATE TABLE mydata (id integer, fruit text, color text, price integer);')
conn.close()

print('Success')

Success


In [5]:
'''
Task 1 Question 2

Write Python code to load “mydata.csv” into “mydata” table.
'''

import csv, sqlite3

conn = sqlite3.connect(r"mydb.db")
cur = conn.cursor()

with open('mydata.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        cur.execute('INSERT INTO mydata (id, fruit, price, color) VALUES (?, ?, ?, ?);', row)

conn.commit()
conn.close()

print('Success')

Success


In [6]:
'''
Task 1 Question 3

Write 2 different SQL statements  with different conditions to retrieve different rows.
Explain which parts of the statement are predicate and which parts are projection.
'''

import csv, sqlite3

conn = sqlite3.connect(r'mydb.db')
cur = conn.cursor()

# 'SELECT AVG(price)' is the projection and 'WHERE fruit = "Grape"' is the predicate
average_price_of_grapes_query = ''' SELECT AVG(price) 
                                    FROM mydata 
                                    WHERE fruit = "Grape"; '''

cur.execute(average_price_of_grapes_query)
result = cur.fetchall()
print('average_price_of_grapes =', result)

# 'SELECT COUNT(id)' is the projection and 'WHERE fruit = "Orange" AND color = "Blue"' is the predicate
count_of_blue_oranges_query = ''' SELECT COUNT(id) 
                                FROM mydata
                                WHERE fruit = "Orange" AND color = "Blue"; '''

cur.execute(count_of_blue_oranges_query)
result = cur.fetchall()
print('count_of_blue_oranges =', result)

conn.close()

average_price_of_grapes = [(54.96414352228306,)]
count_of_blue_oranges = [(41364,)]


In [7]:
'''
Task 2 Question 1

Write Python program that reads “mydata.csv” file and count number of lines.
'''

with open('mydata.csv','r') as f:
    row_count = sum(1 for row in f)
    
row_count

1000001

In [9]:
'''
Task 2 Question 2

By using PyArrow, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatapyarrow.parquet”.
'''

import pyarrow.csv as pa
import pyarrow.parquet as pq

table = pa.read_csv('mydata.csv')
pq.write_to_dataset(table=table,
                    root_path='mydatapyarrow.parquet')

print('Success')

Success


In [12]:
'''
Task 2 Question 3

By using Dask, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatadask.parquet”.
'''

import dask.dataframe as dd

df = dd.read_csv('mydata.csv')
df.to_parquet('mydatadask.parquet')

print('Success')

Success


In [10]:
'''
Task 2 Question 4

By using Pandas, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatapandas.parquet”.
'''

import pandas as pd

df = pd.read_csv('mydata.csv')
df.to_parquet('mydatapandas.parquet')

print('Success')

Success


#### Task 2 Question 5

Examine generated Parquet files.  Why do you think Dask generated Parquet file differently than PyArrow and Pandas? 
What might be explanation for this?

#### Answer

Unlike PyArrow and Pandas, Dask was designed with having to process multiple input files in mind. Dask allows patitioning
the input data into multiple .parquet files named "part.i.parquet" while 'i' stands for the partition number. This
effectively allows Dask to read/write/perform computations on multiple files in parallel in order to improve performance.

Also, when the .parquet files are generated with Dask, additional files are created for metadata storage.

In [13]:
'''
Task 3 Question 1

Write Python code that calculates size of “mydata.csv” in bytes.
Define an integer variable “middle” which is the size of “mydata.csv”  divided by 2. 
'''

import os

middle = os.path.getsize('mydata.csv') / 2

middle

11367204.0

In [16]:
'''
Task 3 Question 2

Write a Python function first_chunk that count number of rows by reading the byte range of the CSV file, 
from 0 till the “middle”. Write a function last_chunk that count number of rows by reading byte range of
CSV file from the “middle”+1 till the end of the file.
'''

import math
import os

def first_chunk(middle):
    with open('mydata.csv','rb') as f:
        byte_stream = f.read(math.floor(middle)).decode(encoding='utf-8')
        return byte_stream.count('\n') if byte_stream[math.floor(middle) - 1] == '\n' else byte_stream.count('\n') + 1 

def last_chunk(middle):
    with open('mydata.csv','rb') as f:
        f.seek(math.floor(middle), 0)
        byte_stream = f.read(math.ceil(middle)).decode(encoding='utf-8')
        return byte_stream.count('\n') - 1 if byte_stream[0] == '\n' else byte_stream.count('\n')

middle = os.path.getsize('mydata.csv') / 2
print('First chunk lines count:', first_chunk(middle))
print("Last chunk lines count:", last_chunk(middle))
print(first_chunk(middle) + last_chunk(middle))

First chunk lines count: 502467
Last chunk lines count: 497535
1000002


#### Task 3 Question 3

Explain why total number of lines from the first chunk and second chunk is larger than
the number of lines calculated in the step (1) of Task 2.

#### Answer

Because for most files the partition will occur right in the middle of a line which as a result will be counted by both first_chunk and last_chunk.

In [15]:
'''
Task 3 Question 4

Suggest an algorithm to resolve the issue from the step (3) and implement it.
'''

'''
The extra line problem occurs when the split is performed in the middle of a line (first chunk doesn't end in '\n'
and last chunk doesn't start with '\n'). We're are going to divide the file size by 2 and then scan the byte stream
until the next newline char and set middle to be the index of the char after that. This way we ensure a line is not
getting split in the middle.

We read more bytes than we wanted in the first chunk, but given that we know the upper length limit for a line in
mydata.csv we want want to do that.
'''

import os

def first_chunk(chunk_size):
    with open('mydata.csv','rb') as f:
        byte_stream = f.read(chunk_size).decode(encoding='utf-8')
        return byte_stream.count('\n')

def last_chunk(chunk_size):
    with open('mydata.csv','rb') as f:
        last_chunk_size = os.path.getsize('mydata.csv') - chunk_size
        f.seek(chunk_size, 0)
        byte_stream = f.read(last_chunk_size).decode(encoding='utf-8')
        # We ignore the first newline char.
        return byte_stream.count('\n')

middle = int(os.path.getsize('mydata.csv') / 2)
with open('mydata.csv','rb') as f:
    f.seek(middle, 0)
    byte_stream = f.read(os.path.getsize('mydata.csv') - middle).decode(encoding='utf-8')
    first_newline = byte_stream.index('\n')
middle_newline = middle + first_newline + 1

print('First chunk lines count:', first_chunk(middle_newline))
print("Last chunk lines count:", last_chunk(middle_newline))
print(first_chunk(middle_newline) + last_chunk(middle_newline))

First chunk lines count: 502467
Last chunk lines count: 497534
1000001


In [17]:
'''
Task 3 Question 5

Check the algorithm of step (4) with multiple chunks. Define a chunk size to be 16MB.
Write a function that process “mydata.csv “ in chunks and count number of lines for
each chunk. For example, first chunk will be 0-16MB, second chunk 16MB-32BM, and so
on, until the last chunk, which might be smaller.
'''

'''
mydata.csv size is only approximately 22MB so we'll try with chunks of 4MB which is more relevant. We'll generalize
the solution of T3Q4 to have another funtion called middle_chunk that returns both started_with_newline and
ended_with_newline.
'''

import os

CHUNK_SIZE = 1024 * 1024 * 4

FILE_SIZE = os.path.getsize('mydata.csv')

def count_lines_in_chunk(seek_to, chunk_size):
    with open('mydata.csv', 'rb') as f:
        f.seek(seek_to, 0)
        byte_stream = f.read(chunk_size).decode(encoding='utf-8')
        return byte_stream.count('\n')

def get_modified_chunk_size(seek_to):
    with open('mydata.csv', 'rb') as f:
        f.seek(seek_to + CHUNK_SIZE, 0)
        byte_stream = f.read(FILE_SIZE - seek_to - CHUNK_SIZE).decode(encoding='utf-8')
        first_newline = byte_stream.index('\n')
    return CHUNK_SIZE + first_newline + 1

total_lines = 0
seek_to = 0
while seek_to <= FILE_SIZE:
    if seek_to + CHUNK_SIZE > FILE_SIZE:
        chunk_lines = count_lines_in_chunk(seek_to, FILE_SIZE - seek_to)
    else:
        modified_chunk_size = get_modified_chunk_size(seek_to)
        chunk_lines = count_lines_in_chunk(seek_to, modified_chunk_size)
    total_lines += chunk_lines
    seek_to += modified_chunk_size

print(total_lines)

1000001
